## Downloading Pre-Trained Model

In [404]:
!wget https://zenodo.org/record/3987831/files/ResNet22_mAP%3D0.430.pth

--2022-03-29 01:28:02--  https://zenodo.org/record/3987831/files/ResNet22_mAP%3D0.430.pth
SSL_INIT
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 259135254 (247M) [application/octet-stream]
Saving to: ‘ResNet22_mAP=0.430.pth’

ResNet22_mAP=0.430. 100%[===================>] 247.13M  2.93MB/s    in 2m 3s   

2022-03-29 01:30:07 (2.02 MB/s) - ‘ResNet22_mAP=0.430.pth’ saved [259135254/259135254]



## Modify JSON

In [1]:
import json
import os

In [3]:
folder = 'result/Thres_Detect/'
if os.path.exists(folder)==False:
    os.mkdir(folder)

folder = 'result/Tuning/Batch'
if os.path.exists(folder)==False:
    os.mkdir(folder)

In [4]:
folder = 'csv_files/Thres-Detect/90/HC/exp-4/85/'
csv_train = os.path.join(folder,'train.csv')
csv_devel = os.path.join(folder,'devel.csv')
csv_test = os.path.join(folder,'test.csv')

dir_input = 'dataset/Thres-Detect/90/HC'
dir_output = 'result/Tuning/Batch/4'

if os.path.exists(dir_output)==False:
    os.mkdir(dir_output)

In [162]:
file_path = 'script/json/config.json'
with open(file_path, 'r+') as f:
    data = json.load(f)
    data['dataset']['train_csv'] = csv_train
    data['dataset']['eval_csv'] = csv_devel
    data['dataset']['test_csv'] = csv_test
    data['dataset']['train_data_root_path'] = dir_input
    data['dataset']['eval_data_root_path'] = dir_input
    data['dataset']['test_data_root_path'] = dir_input
    data['dataset']['class_balancer_batch'] = True
    data['train_config']['logs_path'] = dir_output
    
    data['model']['spec_aug'] = True
    data['data_aumentation']['insert_noise'] = False
    data['model']['mixup'] = True
    data['model']['mixup_alpha'] = 0.5
    
    data['model_name'] = 'panns'
    data['model']['pretreined_checkpoint'] = 'Cnn14_16k_mAP=0.438.pth'
    
    data['audio']['feature'] = 'melspectrogram'
    data['audio']['num_mels'] = 64
    data['audio']['hop_length'] = 320
    
    data['train_config']['optimizer'] = 'adamw'
    data['train_config']['learning_rate'] = 0.001
    data['train_config']['weight_decay'] = 0.01
    data['train_config']['batch_size'] = 4
    
    
    f.seek(0)        # <--- should reset file position to the beginning.
    json.dump(data,f,indent=4)
    f.truncate()     # remove remaining part

In [163]:
data

{'model_name': 'panns',
 'dataset': {'class_balancer_batch': True,
  'window_len': 3,
  'step': 1,
  'padding_with_max_lenght': True,
  'temporal_control': 'padding',
  'max_seq_len': None,
  'train_csv': 'csv_files/Thres-Detect/90/HC/exp-4/85/train.csv',
  'train_data_root_path': 'dataset/Thres-Detect/90/HC',
  'eval_csv': 'csv_files/Thres-Detect/90/HC/exp-4/85/devel.csv',
  'eval_data_root_path': 'dataset/Thres-Detect/90/HC',
  'test_csv': 'csv_files/Thres-Detect/90/HC/exp-4/85/test.csv',
  'test_data_root_path': 'dataset/Thres-Detect/90/HC',
  'control_class': 0,
  'patient_class': 1},
 'model': {'pretreined_checkpoint': 'Cnn14_16k_mAP=0.438.pth',
  'freeze_base': False,
  'mixup': True,
  'mixup_alpha': 0.5,
  'spec_aug': True,
  'num_class': 1},
 'train_config': {'early_stop_epochs': 0,
  'lr_decay': True,
  'warmup_steps': 500,
  'epochs': 100,
  'learning_rate': 0.001,
  'weight_decay': 0.01,
  'optimizer': 'adamw',
  'loss1_weight': 3.0,
  'batch_size': 4,
  'seed': 42,
  'num_

In [164]:
with open(os.path.join(folder,'readme.txt')) as f:
    data = f.read()

print(data)

This File is CSV for Train
Train Data: 
 -Coswara_Positive
 -Coughvid_Positive
 -Compare
 Split:85 - 15


## Training 5 Seeds

In [165]:
import os

config = file_path
seeds = [42]
for seed in seeds:
    command = f"python script/train.py -c {config} -s {seed}"
    os.system(command)

The Max Time dim Lenght is: 64 (+- 1.28 seconds)
The Min Time dim Lenght is: 11 (+- 0.22 seconds)
Using Class Batch Balancer
Enable Mixup with alpha: 0.5
 > Partial model initialization.
 | > Layer missing in the model definition: spectrogram_extractor.stft.conv_real.weight
 | > Layer missing in the model definition: spectrogram_extractor.stft.conv_imag.weight
 | > Layer missing in the model definition: logmel_extractor.melW
 | > 81 / 81 layers are restored.
Partial Pretrained checkpoint loaded:  Cnn14_16k_mAP=0.438.pth
Starting new training run
Write summary at step 10  Loss:  0.491140753030777
Write summary at step 20  Loss:  0.4668610394001007
Write summary at step 30  Loss:  0.4340764582157135
Write summary at step 40  Loss:  1.7025576829910278
Write summary at step 50  Loss:  0.8009523153305054
Write summary at step 60  Loss:  0.18075327575206757
Write summary at step 70  Loss:  1.2606089115142822
Write summary at step 80  Loss:  0.3893384039402008
Write summary at step 90  Loss: 

feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 16

Write summary at step 1140  Loss:  0.5740903615951538
Write summary at step 1150  Loss:  0.8750739097595215
Write summary at step 1160  Loss:  0.4723435044288635
Write summary at step 1170  Loss:  0.6768032312393188
Write summary at step 1180  Loss:  0.5304918885231018
Write summary at step 1190  Loss:  0.4780561923980713
Write summary at step 1200  Loss:  0.4941839575767517
Write summary at step 1210  Loss:  0.6653372049331665
Write summary at step 1220  Loss:  0.6613520383834839
Write summary at step 1230  Loss:  0.29627740383148193
Write summary at step 1240  Loss:  0.6166410446166992
Write summary at step 1250  Loss:  0.7454764246940613
Write summary at step 1260  Loss:  0.9060280323028564
Write summary at step 1270  Loss:  0.5268474221229553
Write summary at step 1280  Loss:  0.5060856938362122
Write summary at step 1290  Loss:  0.5276577472686768
Write summary at step 1300  Loss:  0.8062744736671448
Write summary at step 1310  Loss:  0.4892579913139343
Write summary at step 1320 

feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 16

Write summary at step 2270  Loss:  0.9423403143882751
Write summary at step 2280  Loss:  0.44851231575012207
Write summary at step 2290  Loss:  0.5797465443611145
Write summary at step 2300  Loss:  0.6112965941429138
Write summary at step 2310  Loss:  0.41974127292633057
Write summary at step 2320  Loss:  0.5330371856689453
Write summary at step 2330  Loss:  0.6115337610244751
Write summary at step 2340  Loss:  0.4038950800895691
Write summary at step 2350  Loss:  0.3916601538658142
Write summary at step 2360  Loss:  0.31461411714553833
Write summary at step 2370  Loss:  0.3047546148300171
Write summary at step 2380  Loss:  0.30847758054733276
Write summary at step 2390  Loss:  0.5024015307426453
Write summary at step 2400  Loss:  0.43411847949028015
Write summary at step 2410  Loss:  0.8965754508972168
Write summary at step 2420  Loss:  0.607681393623352
Write summary at step 2430  Loss:  1.0722891092300415
Write summary at step 2440  Loss:  0.8000751733779907
Write summary at step 24

...

feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 16

Write summary at step 35920  Loss:  0.06052244454622269
Write summary at step 35930  Loss:  0.47230738401412964
Write summary at step 35940  Loss:  0.38120490312576294
Write summary at step 35950  Loss:  0.3251712918281555
Write summary at step 35960  Loss:  0.3492065668106079
Write summary at step 35970  Loss:  0.3226924538612366
Write summary at step 35980  Loss:  0.23530030250549316
Write summary at step 35990  Loss:  0.49102577567100525
Write summary at step 36000  Loss:  0.27612680196762085
Saved checkpoint to: result/Tuning/Batch/4/42/panns/checkpoint_36000.pt
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 

feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 16

Write summary at step 37050  Loss:  0.4879416525363922
Write summary at step 37060  Loss:  0.30553317070007324
Write summary at step 37070  Loss:  1.2379989624023438
Write summary at step 37080  Loss:  0.32039278745651245
Write summary at step 37090  Loss:  0.3112693130970001
Write summary at step 37100  Loss:  0.712939441204071
Write summary at step 37110  Loss:  0.08900691568851471
Write summary at step 37120  Loss:  0.6025875806808472
Write summary at step 37130  Loss:  0.09277249127626419
Write summary at step 37140  Loss:  0.30995869636535645
Write summary at step 37150  Loss:  0.8596283197402954
Write summary at step 37160  Loss:  1.0258128643035889
Write summary at step 37170  Loss:  0.3674162030220032
Write summary at step 37180  Loss:  0.39632681012153625
Write summary at step 37190  Loss:  0.36200255155563354
Write summary at step 37200  Loss:  0.19365432858467102
Write summary at step 37210  Loss:  0.34491944313049316
Write summary at step 37220  Loss:  0.3608497977256775
Wr

feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 164, 64])
feature shape input:  torch.Size([10, 16

In [6]:
import os

data = csv_test
folder = dir_input
experiment_dir = dir_output
musan = 'dataset/Musan-Data'
command = f"python script/test_all_seeds.py -t {data} -r {folder} --experiment_dir {experiment_dir}"
os.system(command)

DEBUG: 1
Not K-Fold Experiment
Return Potential: False
Loading checkpoint: result/Tuning/Batch/4/42/panns/best_checkpoint.pt
Model Sucessful Load !
Loading checkpoint: result/Tuning/Batch/4/42/panns/best_checkpoint.pt
Model Sucessful Load !
++++++++++++++++++++++++++++++++++++++++
Control Files Classified incorrectly:
Num. Files: 1
['test_178__1.wav']
++++++++++++++++++++++++++++++++++++++++
----------------------------------------
Patient Files Classified incorrectly:
Num. Files: 12
['test_043__2.wav' 'test_046__0.wav' 'test_046__1.wav' 'test_046__2.wav'
 'test_067__0.wav' 'test_067__1.wav' 'test_067__2.wav' 'test_092__0.wav'
 'test_092__1.wav' 'test_103__2.wav' 'test_108__3.wav' 'test_184__1.wav']
----------------------------------------
======== Confusion Matrix ==========
Predicted  0.0  1.0  All
Target                  
0          120    1  121
1           12   21   33
All        132   22  154
Test
  Acurracy:  0.9155844155844156 Acurracy Control:  0.9917355371900827 Acurracy Pati

0

In [167]:
import glob

files = glob.glob(os.path.join(dir_output,'*/*/*.pt'))

for file in files:
    filename = file.split('/')[-1]
    if filename.split('_')[0] == 'checkpoint':
        os.remove(file)